In [4]:
import os
import pandas as pd
import logging
import re
import ast
import numpy as np 
import torch
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
from sentence_transformers import SentenceTransformer

from concurrent.futures import ThreadPoolExecutor
from collections import Counter, defaultdict
from seqeval.metrics import classification_report, f1_score

from wtpsplit import SaT

sat = SaT("sat-12l", language="fr", style_or_domain="ud")

In [21]:
df = pd.read_csv("../Temporal_Entities_Detection/data/df_extended_intersection.csv")
df

,article_id,segment_id,segment_text,gold_annotations,publication_date,heideltime_dates_with_spans
0,0,0,Insécurité alimentaire en milieu urbain : La v...,"[{'label': 'departement', 'text': 'Ouagadougou...",2009-01-15,"[{'date': '2009-01-15', 'span': [74, 95]}, {'d..."
1,0,1,pour mesurer et caractériser la vulnérabilité ...,"[{'label': 'departement', 'text': 'Ouagadougou...",2009-01-15,"[{'date': '2007', 'span': [345, 349]}]"
2,1,0,Electrification : Il n’y a pas d’eau pour le c...,"[{'label': 'village', 'text': 'Kompienga', 'st...",2009-08-03,"[{'date': '2009-08-03', 'span': [53, 70]}, {'d..."
3,1,1,15 à 20 pour cent de la consommation du CRCO. ...,"[{'label': 'departement', 'text': 'Bagré', 'st...",2009-08-03,"[{'date': 'PRESENT_REF', 'span': [192, 204]}, ..."
4,2,0,Prévision pluviométrique des mois de juillet-a...,"[{'label': 'departement', 'text': 'Ouagadougou...",2009-06-05,"[{'date': '2009-07', 'span': [37, 44]}, {'date..."
...,...,...,...,...,...,...
3440,998,3,que l’opération devait se mener sur deux axes ...,"[{'label': 'departement', 'text': 'Tougan', 's...",2009-11-09,"[{'date': '2005-01-26', 'span': [886, 904]}]"
3441,998,4,en l’abandonnant sur les lieux sans informer p...,"[{'label': 'village', 'text': 'Ouakara', 'star...",2009-11-09,"[{'date': '2003-08-03', 'span': [1235, 1249]}]"
3442,998,5,"Son intention, dit-il, n’était pas de lui donn...","[{'label': 'departement', 'text': 'Dédougou', ...",2009-11-09,[]
3443,999,0,AGRESSION D’UN HUISSIER DE JUSTICE : Appolinai...,"[{'label': 'departement', 'text': 'Ouagadougou...",2009-03-17,"[{'date': '2009-03-17', 'span': [92, 110]}, {'..."


In [22]:
import ast 

def reconstruire_articles(df: pd.DataFrame) -> pd.DataFrame:
    articles_reconstruits = []
    
    for article_id, group in df.groupby('article_id'):
        group = group.sort_values('segment_id')
        
        texte_complet = ""
        offset_courant = 0
        annotations_lieux = []
        annotations_dates = []
        
        for idx, row in group.iterrows():
            segment_text = row['segment_text']
            
            processed_text, insertion_positions = process_segment(segment_text)
            
            if texte_complet:
                texte_complet += " "
                offset_courant += 1
            
            debut_segment = offset_courant
            texte_complet += processed_text
            longueur_segment = len(processed_text)
            
            # Conversion et traitement des annotations de lieux
            gold_annotations = row['gold_annotations']
            if isinstance(gold_annotations, str):
                try:
                    gold_annotations = ast.literal_eval(gold_annotations)
                except:
                    gold_annotations = []
            
            if gold_annotations:
                for annot in gold_annotations:
                    annot_mod = annot.copy() if isinstance(annot, dict) else dict(annot)
                    orig_start = annot_mod['start']
                    orig_end = annot_mod['end']
                    new_start, new_end = update_span(orig_start, orig_end, insertion_positions)
                    annot_mod['start'] = new_start + debut_segment
                    annot_mod['end'] = new_end + debut_segment
                    annotations_lieux.append(annot_mod)
            
            # Conversion et traitement des annotations de dates
            heideltime_dates = row['heideltime_dates_with_spans']
            if isinstance(heideltime_dates, str):
                try:
                    heideltime_dates = ast.literal_eval(heideltime_dates)
                except:
                    heideltime_dates = []
            
            if heideltime_dates:
                for annot in heideltime_dates:
                    annot_mod = annot.copy() if isinstance(annot, dict) else dict(annot)
                    orig_span = annot_mod.get('span', (0, 0))
                    new_span_start, new_span_end = update_span(orig_span[0], orig_span[1], insertion_positions)
                    annot_mod['span'] = (new_span_start + debut_segment, new_span_end + debut_segment)
                    annotations_dates.append(annot_mod)
            
            offset_courant += longueur_segment
        
        article_reconstruit = {
            'article_id': article_id,
            'texte_complet': texte_complet,
            'gold_annotations': annotations_lieux,
            'heideltime_dates_with_spans': annotations_dates,
            'publication_date': group.iloc[0]['publication_date']
        }
        articles_reconstruits.append(article_reconstruit)
    
    return pd.DataFrame(articles_reconstruits)

In [ ]:
reconstruct_df_new = reconstruire_articles(df)
reconstruct_df_new

,article_id,texte_complet,gold_annotations,heideltime_dates_with_spans,publication_date
0,0,Insécurité alimentaire en milieu urbain : La v...,"[{'label': 'departement', 'text': 'Ouagadougou...","[{'date': '2009-01-15', 'span': (75, 96)}, {'d...",2009-01-15
1,1,Electrification : Il n’y a pas d’eau pour le c...,"[{'label': 'village', 'text': 'Kompienga', 'st...","[{'date': '2009-08-03', 'span': (54, 71)}, {'d...",2009-08-03
2,2,Prévision pluviométrique des mois de juillet-a...,"[{'label': 'departement', 'text': 'Ouagadougou...","[{'date': '2009-07', 'span': (37, 44)}, {'date...",2009-06-05
3,3,"JEROME BOUGOUMA, MINISTRE EN CHARGE DU TRAVAIL...","[{'label': 'departement', 'text': 'Ouagadougou...","[{'date': '2009-05-25', 'span': (102, 119)}, {...",2009-05-25
4,4,Lutte contre les maladies tropicales négligées...,"[{'label': 'village', 'text': 'Gangaol', 'star...","[{'date': '2009', 'span': (80, 84)}, {'date': ...",2009-01-28
...,...,...,...,...,...
995,995,"PRISE D’OTAGES EN MAURITANIE : Le Burkina, vic...","[{'label': 'country', 'text': 'Burkina', 'star...","[{'date': '2009-12-21', 'span': (64, 86)}, {'d...",2009-12-21
996,996,Lutte contre le grand banditisme : La gendarme...,"[{'label': 'departement', 'text': 'Bagré', 'st...","[{'date': '2009-07-02', 'span': (138, 158)}, {...",2009-07-02
997,997,Fait divers : Un bébé dans une fosse septique ...,"[{'label': 'village', 'text': 'Manga', 'start'...","[{'date': '2009-03-16', 'span': (72, 90)}, {'d...",2009-03-16
998,998,ASSISES CRIMINELLES DE DEDOUGOU : Prison à vie...,"[{'label': 'village', 'text': 'Passakongo', 's...","[{'date': '2009-11-09', 'span': (79, 100)}, {'...",2009-11-09


In [24]:
lexique_corrected = pd.read_csv("data/lexique_enrichi_final_july.csv")
lexique_corrected

,term,concept,theme,phase
0,baisse de la qualité du sol,champ infertile,agriculture,NaN
1,champ infertile,champ infertile,agriculture,NaN
2,dégradation du sol,champ infertile,agriculture,NaN
3,incapable de semer,champ infertile,agriculture,NaN
4,infertilité du sol,champ infertile,agriculture,NaN
...,...,...,...,...
428,énorme explosion,violence,sociopolitique,NaN
429,groupes vulnerables,vulnérabilité,sociopolitique,precrise
430,personnes exposées,vulnérabilité,sociopolitique,precrise
431,personnes vulnerables,vulnérabilité,sociopolitique,precrise


In [3]:
lexique_corrected

,term,concept,theme,phase
0,baisse de la qualité du sol,champ infertile,agriculture,NaN
1,champ infertile,champ infertile,agriculture,NaN
2,dégradation du sol,champ infertile,agriculture,NaN
3,incapable de semer,champ infertile,agriculture,NaN
4,infertilité du sol,champ infertile,agriculture,NaN
...,...,...,...,...
428,énorme explosion,violence,sociopolitique,NaN
429,groupes vulnerables,vulnérabilité,sociopolitique,precrise
430,personnes exposées,vulnérabilité,sociopolitique,precrise
431,personnes vulnerables,vulnérabilité,sociopolitique,precrise


In [ ]:
def compute_sentence_embeddings(text):
    sentences = sat.split(text, do_paragraph_segmentation=False)
    return [phrase for phrase in sentences if phrase]


reconstruct_df_new['sentences'] = reconstruct_df_new['texte_complet'].apply(compute_sentence_embeddings)

In [ ]:
import pandas as pd
import re
from datetime import datetime
from typing import List, Dict, Optional, Tuple, Set

# Liste des termes vagues à traiter spécialement
VAGUE_TERMS = ['production', 'campagne', 'prix', 'coût', 'crise', 'stock', 'pluie', 'pluiviométrie']

# Définition des triggers d'augmentation et de diminution
# 'plus', 'meilleur', 'meilleure', 'supérieur', 'supérieure'
# 'renforcement', 'renforce', 'renforcent', 'renforcer'
AUGMENTATION_TRIGGERS = [
    'augmentation', 'augmenté', 'augmente', 'augmentent', 'augmenter', 'augmentée', 'augmentées',
    'hausse', 'hausse', 'haussier', 'haussière', 'hausse', 'haussent',
    'accroissement', 'accroître', 'accroît', 'accru', 'accrue',
    'progression', 'progresse', 'progressent', 'progresser',
    'croissance', 'croît', 'croissant', 'croissante',
    'amélioration', 'améliore', 'améliorent', 'améliorer',
    'montée', 'monte', 'montent', 'monter',
    'élévation', 'élève', 'élèvent', 'élever',
    'intensification', 'intensifie', 'intensifient', 'intensifier'
]
DIMINUTION_TRIGGERS = [
    'diminution', 'diminué', 'diminue', 'diminuent', 'diminuer',
    'baisse', 'baissé', 'baisse', 'baissent', 'baisser',
    'réduction', 'réduit', 'réduite', 'réduisent', 'réduire',
    'chute', 'chuté', 'chute', 'chutent', 'chuter',
    'déclin', 'décline', 'déclinent', 'décliner',
    'recul', 'recule', 'reculent', 'reculer',
    'régression', 'régresse', 'régressent', 'régresser',
    'dégradation', 'dégrade', 'dégradent', 'dégrader',
    'détérioration', 'détériore', 'détériorent', 'détériorer',
    'affaiblissement', 'affaibli', 'affaiblie', 'affaiblissent', 'affaiblir',
    'moins', 'pire', 'inférieur', 'inférieure', 'moindre'
]

ALL_TRIGGERS = set(AUGMENTATION_TRIGGERS + DIMINUTION_TRIGGERS)
def generate_term_variations(term: str) -> List[str]:
    variations = [
        term,  
        term + 's',  
        term + 'es',  
        term + "'",  
    ]
    return variations

def generate_trigger_variations() -> Set[str]:
    all_variations = set()
    for trigger in ALL_TRIGGERS:
        variations = generate_term_variations(trigger)
        for var in variations:
            all_variations.add(var.lower())
    return all_variations

# Initialisation des variations de triggers
TRIGGER_VARIATIONS = generate_trigger_variations()

class PhraseContext:
    def __init__(self, text: str, terms: List[Dict], locations: List[Dict], dates: List[Dict]):
        self.text = text
        self.terms = terms
        self.locations = locations
        self.dates = dates
        
    @property
    def has_terms(self) -> bool:
        return len(self.terms) > 0
    
    @property
    def has_entities(self) -> bool:
        return len(self.locations) > 0 or len(self.dates) > 0
    
    def has_non_vague_terms(self) -> bool:
        """Vérifie s'il existe des termes non vagues dans le contexte"""
        return any(not term['is_vague'] for term in self.terms)
    
    def has_trigger_terms(self) -> bool:
        """Vérifie s'il existe des termes de triggers dans le contexte"""
        # Vérification de la présence de mots triggers dans le texte
        words = re.findall(r'\b\w+\b', self.text.lower())
        return any(word in TRIGGER_VARIATIONS for word in words)

def get_location_hierarchy_rank(label: str) -> int:
    """Définit la hiérarchie des lieux du plus petit au plus grand."""
    hierarchy = {
        'village': 1,
        'departement': 2,
        'province': 3,
        'region': 4,
        'country': 5
    }
    return hierarchy.get(label, 0)

def find_closest_date(dates: List[Dict], publication_date: str) -> Optional[Dict]:
    if not dates:
        return None
    try:
        pub_date_dt = datetime.strptime(publication_date, "%Y-%m-%d")
    except ValueError:
        return dates[0]
    
    valid_dates = []
    for d in dates:
        try:
            dt = datetime.strptime(d['date'], "%Y-%m-%d")
            if dt <= pub_date_dt:
                valid_dates.append(d)
        except ValueError:
            pass
    
    if not valid_dates:
        return None
    
    closest_dt = min(valid_dates, key=lambda x: abs(datetime.strptime(x['date'], "%Y-%m-%d") - pub_date_dt))
    return closest_dt

def find_terms(phrase: str, lexique_df: pd.DataFrame) -> List[Dict]:
    term_variations = {}
    for term in lexique_df['term'].str.lower().unique():
        # Pour chaque terme, générer ses variations
        variations = generate_term_variations(term)
        for variation in variations:
            term_variations[variation] = term
    
    # Créer le pattern regex
    pattern = r'\b(' + '|'.join(map(re.escape, term_variations.keys())) + r')\b'
    matches = re.finditer(pattern, phrase.lower())
    results_terms = []
    
    for m in matches:
        base_variation = m.group(1).lower()
        # Retrouver le terme original
        base_term = term_variations[base_variation]
        
        # Rechercher dans le lexique
        original_rows = lexique_df[lexique_df['term'].str.lower() == base_term]
        
        if original_rows.empty:
            continue
        
        original_row = original_rows.iloc[0].to_dict()
        
        # Préparation de l'entrée de terme
        term_entry = {
            'term': original_row['term'],  # Le terme original du lexique
            'concept': original_row['concept'],
            'theme': original_row['theme'],
            'phase': original_row['phase'],  # Ajout de la phase
            'position': (m.start(), m.end()),
            'is_vague': base_term.lower() in VAGUE_TERMS
        }
        
        results_terms.append(term_entry)
    
    return results_terms

def create_result_entry(article_id: str, term_info: Dict, location: Optional[Dict], 
                       date: Optional[Dict], phrase: str, contexte_enrichi: str,
                       companion_term_info: Optional[Dict] = None) -> Dict:
    
    # Gestion des termes et de leurs concepts
    if companion_term_info:
        # Si les concepts sont différents, on les concatène
        if term_info['concept'] != companion_term_info['concept']:
            combined_concept = f"{term_info['concept']} + {companion_term_info['concept']}"
        else:
            combined_concept = term_info['concept']
        
        # Même chose pour les thèmes
        if term_info['theme'] != companion_term_info['theme']:
            combined_theme = f"{term_info['theme']} + {companion_term_info['theme']}"
        else:
            combined_theme = term_info['theme']
        
        # Concaténation des termes
        combined_term = f"{term_info['term']}, {companion_term_info['term']}"
        
        # Gestion de la phase - prendre la phase du terme principal
        phase = term_info.get('phase', None)
    else:
        combined_concept = term_info['concept']
        combined_theme = term_info['theme']
        combined_term = term_info['term']
        phase = term_info.get('phase', None)
        
    result_entry = {
        'article_id': article_id,
        'term': combined_term,
        'concept': combined_concept,
        'theme': combined_theme,
        'phase': phase,  
        'location': location,
        'date': date['date'] if date else None,
        'date_span': date['span'] if date else None,
        'phrase': phrase,  # Phrase originale
        'contexte_enrichi': contexte_enrichi,  # Nouveau champ pour le contexte enrichi
        'complete_triplet': bool(location and date)
    }
    return result_entry

def count_sentences(text: str) -> int:
    """Compte approximativement le nombre de phrases dans un texte."""
    # Cette fonction est simple mais pourrait être améliorée selon la complexité réelle de vos phrases
    sentences = re.split(r'[.!?]+', text.strip())
    # Filtrer les chaînes vides
    sentences = [s for s in sentences if s.strip()]
    return len(sentences)

# def get_context_sentences(all_phrases: List[str], current_index: int, phrase: str) -> str:
#     """
#     Crée un contexte enrichi en ajoutant des phrases avant et après la phrase courante.
#     Si la phrase contient une seule phrase: ajoute 2 avant et 2 après
#     Si la phrase contient plusieurs phrases: ajoute 1 avant et 1 après
#     """
#     # Estimer si la phrase courante contient une ou plusieurs phrases
#     sentence_count = count_sentences(phrase)
    
#     if sentence_count <= 1:
#         # Pour une seule phrase, ajouter 2 avant et 2 après
#         before_count = 2
#         after_count = 2
#     else:
#         # Pour plusieurs phrases, ajouter 1 avant et 1 après
#         before_count = 1
#         after_count = 1
    
#     # Calculer les indices de début et de fin
#     start_idx = max(0, current_index - before_count)
#     end_idx = min(len(all_phrases), current_index + after_count + 1)
    
#     # Construire le contexte enrichi
#     context_sentences = all_phrases[start_idx:end_idx]
#     enriched_context = " ".join(context_sentences)
    
#     return enriched_context

def get_context_sentences(all_phrases: List[str], current_index: int, phrase: str) -> str:
    """
    Crée un contexte enrichi en ajoutant une phrase avant et une phrase après la phrase courante.
    """
    # Toujours ajouter 1 phrase avant et 1 phrase après, peu importe le contenu de la phrase courante
    before_count = 1
    after_count = 1
    
    # Calculer les indices de début et de fin
    start_idx = max(0, current_index - before_count)
    end_idx = min(len(all_phrases), current_index + after_count + 1)  # +1 car on inclut l'index courant
    
    # Construire le contexte enrichi
    context_sentences = all_phrases[start_idx:end_idx]
    enriched_context = " ".join(context_sentences)
    
    return enriched_context

def process_reconstructed_articles(reconstruct_df: pd.DataFrame, lexique_df: pd.DataFrame) -> pd.DataFrame:
    results = []
    # Pour chaque article reconstruit
    for idx, row in reconstruct_df.iterrows():
        article_id = row['article_id']
        full_text = row['texte_complet']
        publication_date = row.get('publication_date', row.get('annee'))
        
        # Segmentation en phrases avec SaT
        all_phrases = row['sentences']
        all_phrases = [phrase.strip() for phrase in all_phrases if phrase.strip()]
        
        # Construction des contextes de phrases
        phrase_contexts = []
        pos = 0
        for phrase in all_phrases:
            if not phrase:
                continue
            
            phrase_start = full_text.find(phrase, pos)
            if phrase_start == -1:
                continue
                
            phrase_end = phrase_start + len(phrase)
            pos = phrase_end
            
            # Extraction des annotations spatiales et temporelles
            current_locations = []
            if isinstance(row['gold_annotations'], list):
                for loc in row['gold_annotations']:
                    if loc['start'] >= phrase_start and loc['end'] <= phrase_end:
                        if not any(existing_loc['text'] == loc['text'] for existing_loc in current_locations):
                            current_locations.append(loc)
            
            current_dates = []
            if isinstance(row['heideltime_dates_with_spans'], list):
                for date_ann in row['heideltime_dates_with_spans']:
                    local_start, local_end = date_ann.get('span', (0, 0))
                    if local_start >= phrase_start and local_end <= phrase_end:
                        if not any(d['date'] == date_ann['date'] for d in current_dates):
                            current_dates.append({
                                'date': date_ann['date'],
                                'span': date_ann['span']
                            })
            
            # Recherche des termes dans la phrase
            current_terms = find_terms(phrase, lexique_df)
            
            phrase_contexts.append(PhraseContext(
                text=phrase,
                terms=current_terms,
                locations=current_locations,
                dates=current_dates
            ))
        
        # Projections spatiales et temporelles
        num_phrases = len(phrase_contexts)
        spatial_proj = [None] * num_phrases
        
        for i, ctx in enumerate(phrase_contexts):
            if ctx.locations:
                spatial_proj[i] = min(ctx.locations, key=lambda x: get_location_hierarchy_rank(x['label']))
        
        for i in range(num_phrases):
            if spatial_proj[i] is not None:
                for j in [i+1, i+2]:
                    if j < num_phrases and spatial_proj[j] is None:
                        spatial_proj[j] = spatial_proj[i]
        
        default_spatial = {'label': 'country', 'text': 'Burkina'}
        for i in range(num_phrases):
            if spatial_proj[i] is None:
                spatial_proj[i] = default_spatial
        
        temporal_proj = [None] * num_phrases
        last_temp = None
        
        for i, ctx in enumerate(phrase_contexts):
            if ctx.dates:
                best_date = find_closest_date(ctx.dates, publication_date)
                temporal_proj[i] = best_date
                last_temp = best_date
            else:
                temporal_proj[i] = last_temp
        
        for i in range(num_phrases):
            if temporal_proj[i] is None:
                temporal_proj[i] = {'date': publication_date, 'span': (0, 0)}
        
        # Nouvelle logique de traitement des termes
        for i, current_context in enumerate(phrase_contexts):
            if current_context.has_terms:
                current_best_location = spatial_proj[i]
                current_best_date = temporal_proj[i]
                original_phrase = current_context.text
                
                # Enrichir la phrase avec les phrases contextuelles
                contexte_enrichi = get_context_sentences(all_phrases, i, original_phrase)
                
                for j, term_info in enumerate(current_context.terms):
                    # Traitement spécial pour les termes vagues
                    if term_info['is_vague']:
                        # Chercher un terme non vague dans le contexte local
                        companion_term = next((t for t in current_context.terms 
                                               if not t['is_vague'] and t['term'] != term_info['term']), None)
                        
                        # Si pas de terme accompagnant dans le contexte local, on vérifie les triggers
                        if not companion_term:
                            # Vérifier si un trigger d'augmentation ou de diminution est présent
                            if not current_context.has_trigger_terms():
                                # Pas de trigger, on ignore ce terme vague
                                continue
                            # Sinon, on garde le terme vague sans terme accompagnant
                            result_entry = create_result_entry(
                                article_id=article_id,
                                term_info=term_info,
                                location=current_best_location,
                                date=current_best_date,
                                phrase=original_phrase,  # Phrase originale
                                contexte_enrichi=contexte_enrichi  # Contexte enrichi dans une colonne séparée
                            )
                        else:
                            # Cas normal avec terme accompagnant
                            result_entry = create_result_entry(
                                article_id=article_id,
                                term_info=term_info,
                                location=current_best_location,
                                date=current_best_date,
                                phrase=original_phrase,  # Phrase originale
                                contexte_enrichi=contexte_enrichi,  # Contexte enrichi dans une colonne séparée
                                companion_term_info=companion_term
                            )
                    else:
                        # Pour les termes non vagues, comportement standard
                        result_entry = create_result_entry(
                            article_id=article_id,
                            term_info=term_info,
                            location=current_best_location,
                            date=current_best_date,
                            phrase=original_phrase,  # Phrase originale
                            contexte_enrichi=contexte_enrichi  # Contexte enrichi dans une colonne séparée
                        )
                    
                    # Vérification des doublons
                    is_duplicate = any(
                        existing_result['article_id'] == result_entry['article_id'] and
                        existing_result['term'] == result_entry['term'] and
                        existing_result['location'] == result_entry['location'] and
                        existing_result['date'] == result_entry['date'] and 
                        existing_result['date_span'] == result_entry['date_span']
                        for existing_result in results
                    )
                    
                    if not is_duplicate:
                        results.append(result_entry)
    
    return pd.DataFrame(results)
